The World Health Organization (WHO) undertook the development of a rapid guide on the use of chest imaging in the diagnosis and management of coronavirus disease 2019 (COVID-19). The rapid guide was developed over 2 months by using standard WHO processes, except for the use of “rapid reviews” and online meetings of the panel. The evidence review was supplemented by a survey of stakeholders regarding their views on the acceptability, feasibility, impact on equity, and resource use of the relevant chest imaging modalities (chest radiography, chest CT, and lung US). The guideline development group had broad expertise and country representation. The rapid guide includes three diagnosis recommendations and four management recommendations. The recommendations cover patients with confirmed or who are suspected of having COVID-19 with different levels of disease severity, throughout the care pathway from outpatient facility or hospital entry to home discharge. All recommendations are conditional and are based on low certainty evidence (n = 2), very low certainty evidence (n = 2), or expert opinion (n = 3). The remarks accompanying the recommendations suggest which patients are likely to benefit from chest imaging and what factors should be considered when choosing the specific imaging modality. The guidance offers considerations about implementation, monitoring, and evaluation, and also identifies research needs.

In [ ]:

!conda install gdcm -c conda-forge -y
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
import glob
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
from skimage import exposure
#import gdcm
import cv2
import warnings
from fastai.vision.all import *
from fastai.medical.imaging import *
warnings.filterwarnings('ignore')
dataset_path = Path('../input/siim-covid19-detection')
import vtk
# numba
import numba
from numba import jit
from vtk.util import numpy_support


In [ ]:
def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='gray', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:

def image_path(row):
    study_path = dataset_path/'train'/row.StudyInstanceUID
    for i in get_dicom_files(study_path):
        if row.id.split('_')[0] == i.stem: return i 
        



class Config:
    n_folds: int = 5
    seed: int = 2021
    num_classes: int = 2 
    img_size: int = 256
    fold_num: int = 0
    device: str = 'cuda:0'


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(Config.seed)

In [ ]:
# Thanks https://www.kaggle.com/tanlikesmath/siim-covid-19-detection-a-simple-eda
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
        

In [ ]:
dicom_paths = get_dicom_files(dataset_path/'train')
imgs = [dicom2array(path) for path in dicom_paths[:4]]
plot_imgs(imgs)


In [ ]:
train_image_df = pd.read_csv(dataset_path/'train_image_level.csv')


# Thanks https://www.kaggle.com/tanlikesmath/siim-covid-19-detection-a-simple-eda
train_image_df['class'] = train_image_df.label.apply(lambda x: x.split()[0])


train_image_df['x_min'] = train_image_df.label.apply(lambda x: float(x.split()[2]))
train_image_df['y_min'] = train_image_df.label.apply(lambda x: float(x.split()[3]))
train_image_df['x_max'] = train_image_df.label.apply(lambda x: float(x.split()[4]))
train_image_df['y_max'] = train_image_df.label.apply(lambda x: float(x.split()[5]))



def get_bbox_area(row):
    return (row['x_max']-row['x_min'])*(row['y_max']-row['y_min'])


train_image_df['bbox_area'] = train_image_df.apply(get_bbox_area, axis=1)

train_image_df['image_path'] = train_image_df.apply(image_path, axis=1)

train_image_df['bbox_area'].hist()

In [ ]:
train_image_df.tail()

In [ ]:
thickness = 3
scale = 5

imgs = []
image_paths = train_image_df['image_path'].values
class_ids = train_image_df['class']

# map label_id to specify color
label2color = {class_id:[random.randint(0,255) for i in range(3)] for class_id in class_ids}
for i in range(8):
    image_path = random.choice(image_paths)
    print(image_path)
    img = dicom2array(str(image_path))
    print(img)

    img = cv2.resize(img, (224,224), fx=1/scale, fy=1/scale, interpolation = cv2.INTER_CUBIC)
    img = np.stack([img, img, img], axis=-1)
    
    box = train_image_df.loc[train_image_df['image_path'] == image_path, ['x_min', 'y_min', 'x_max', 'y_max']].values[0]/scale
    label = train_image_df.loc[train_image_df['image_path'] == image_path, ['class']].values[0][0]
    
    color = label2color[label]
    img = cv2.rectangle(
        img,
        (int(box[0]), int(box[1])),
        (int(box[2]), int(box[3])),
        color, thickness)
    
    img = cv2.resize(img, (500,500))
    imgs.append(img)
    
plot_imgs(imgs, cmap=None)